In [ ]:
import os
import sys

import numpy as np
import cv2
from matplotlib import pyplot as plt
import importlib

sys.path.append("../")

import config

from model_training import training_data_loader

In [ ]:
training_data = training_data_loader.load_data_for_labeled_batches(labeled_batch_dirs=training_data_loader.LABELED_BATCH_DIRS_1)
average_number_of_persons = sum([len(fc) for fc in training_data.centre_points]) / len(training_data.centre_points)
print(f"Number of annotate frames: {len(training_data.centre_points)}, "
      f"with average number of persons on a frame = {average_number_of_persons:.2f}")

In [ ]:
frame_index = 0

In [ ]:
# training_data.flip_vertically()
# training_data.flip_horizontally()

In [ ]:
ir_frame = training_data.normalized_ir_data[frame_index]
plt.imshow(ir_frame)

In [ ]:
# ir_frame
list(range(1, 2))

In [ ]:
def draw_airbrush_circle(img, centre, radius):
    for x in range(max(0, round(centre[0]-radius)), min(img.shape[0], round(centre[0]+radius+1))):
        for y in range(max(0, round(centre[1]-radius)), min(img.shape[1], round(centre[1]+radius+1))):
            point = (x, y)
            distance_to_centre = cv2.norm((centre[0] - x, centre[1] - y))
            if distance_to_centre > radius:
                continue
            img[point] = max(img[point], 1 - distance_to_centre / radius)
    
    
def get_img_reconstructed_from_labels(frame_index):
    centre_points = training_data.centre_points[frame_index]

    SCALE_FACTOR = 5
    img_reconstructed_scaled = np.zeros(shape=(config.IR_CAMERA_RESOLUTION[0] * SCALE_FACTOR, 
                                               config.IR_CAMERA_RESOLUTION[1] * SCALE_FACTOR))

    for centre_point in centre_points:
        centre_point = centre_point[::-1]  # reversed x and y in 

        draw_airbrush_circle(img=img_reconstructed_scaled, 
                             centre=[p*SCALE_FACTOR for p in centre_point], 
                             radius=int(5*SCALE_FACTOR))

    # plt.imshow(img_reconstructed_scaled)

    img_reconstructed_original_size = cv2.resize(
        src=img_reconstructed_scaled, 
        dsize=config.IR_CAMERA_RESOLUTION_XY, 
        interpolation=cv2.INTER_LINEAR)
        
    plt.imshow(img_reconstructed_original_size)
    return img_reconstructed_original_size
        

img = get_img_reconstructed_from_labels(0)

In [ ]:
img16 = (img * 8).astype('int')
plt.imshow(img16)
img16